# ConTexto

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(url_search):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    h2_results = soup.find_all('h2', {'class': 'title'})
    a_links = []
    for item in h2_results:
        try:
            a_links.append(item.find('a')['href'])
        except:
            pass
    return a_links

In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(url_search)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links

In [5]:
url_search_text = 'https://seer.ufrgs.br/index.php/ConTexto/search/search?query=__query__&searchJournal=3&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=score&orderDir=desc&searchPage=__pg_num__#results'

listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)


['https://seer.ufrgs.br/index.php/ConTexto/article/view/47255',
 'https://seer.ufrgs.br/index.php/ConTexto/article/view/68040',
 'https://seer.ufrgs.br/index.php/ConTexto/article/view/94641',
 'https://seer.ufrgs.br/index.php/ConTexto/article/view/96721',
 'https://seer.ufrgs.br/index.php/ConTexto/article/view/41595',
 'https://seer.ufrgs.br/index.php/ConTexto/article/view/35725',
 'https://seer.ufrgs.br/index.php/ConTexto/article/view/38072',
 'https://seer.ufrgs.br/index.php/ConTexto/article/view/36999',
 'https://seer.ufrgs.br/index.php/ConTexto/article/view/108673',
 'https://seer.ufrgs.br/index.php/ConTexto/article/view/99437',
 'https://seer.ufrgs.br/index.php/ConTexto/article/view/93779',
 'https://seer.ufrgs.br/index.php/ConTexto/article/view/30340']

In [6]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [7]:
def captura_titulo(soup):
    try:
        title = soup.find('h1', {'class': 'page_title'}).text
        title = ' '.join(title.split())
    except:
        title = ''
    return {'Título': title}

In [8]:
def captura_data(soup):
    try:
        date_tag = soup.find('div', {'class': 'item published'})
        date = date_tag.find('div', {'class': 'value'}).text
        date = ' '.join(date.split())
    except:
        date = ''
    return {'Data de Publicação': date}

In [9]:
def captura_resumo(soup):
    try:
        abstract = soup.find('section', {'class': 'item abstract'}).text
        abstract = abstract.replace('Resumo', '')
        abstract = ' '.join(abstract.split())
    except:
        abstract = ''
    return {'Resumo': abstract}

In [10]:
def captura_palavras_chave(soup):
    try:
        kw_tag = soup.find('section', {'class': 'item keywords'})
        kw = kw_tag.find('span', {'class': 'value'}).text.replace(',', '.').replace(';', '.')
        kw = ' '.join(kw.split())
    except:
        kw = ''
    return {'Palavras-chave': kw}

In [11]:
def captura_autores(soup):
    autores = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        autores = authors_ul.find_all('span', {'class': 'name'})
        autores = [' '.join(a.text.split()) for a in autores]
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [12]:
def captura_afiliacao(soup):
    afiliacao = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        afiliacao = authors_ul.find_all('span', {'class': 'affiliation'})
        afiliacao = [' '.join(a.text.split()) for a in afiliacao]
        
    except:
        pass
    afiliacao = completar_lista(afiliacao)
    
    key_affiliation = [f"Afiliação {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, afiliacao))

In [13]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'ConTexto'}
    url_search_text = 'https://seer.ufrgs.br/index.php/ConTexto/search/search?query=__query__&searchJournal=3&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=score&orderDir=desc&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y-%m-%d')
    return df

## Pesquisa por artigos

In [14]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head(3)

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2016-03-18,ANÁLISE DE SUSTENTABILIDADE AMBIENTAL: ESTUDO ...,As organizações conquistam desenvolvimento sus...,Indústria de eletrodomésticos. Sustentabilidad...,4,Daniela Di Domenico,Sady Mazzioni,Maicon Andre Provin,Luciano Toniello,,,Universidade Comunitária da Região de Chapecó ...,Universidade Comunitária da Região de Chapecó ...,Universidade Comunitária da Região de Chapecó ...,Universidade Comunitária da Região de Chapecó ...,,,https://seer.ufrgs.br/index.php/ConTexto/artic...,ConTexto
1,2019-06-27,MAPEAMENTO DA PESQUISA SOBRE DISCLOSURE AMBIEN...,O objetivo desse trabalho é apresentar uma rev...,Disclosure ambiental. Revisão da literatura. R...,2,Márcio Abadio Ramos,Rodrigo Fernandes Malaquias,,,,,Universidade Federal de Uberlândia.,Universidade Federal de Uberlândia,,,,,https://seer.ufrgs.br/index.php/ConTexto/artic...,ConTexto
2,2019-11-25,ANÁLISE DAS DEMONSTRAÇÕES CONTÁBEIS: UMA PROPO...,Este trabalho apresenta os intervalos dos indi...,Demonstrações Contábeis. Análise. Índices de L...,3,Paula Hortência da Silva Soares,Juliano Almeida de Faria,José Jackson de Oliveira,,,,Universidade Estadual de Feira de Santana Aven...,Universidade Estadual de Feira de Santana Aven...,Universidade Estadual de Feira de Santana Univ...,,,,https://seer.ufrgs.br/index.php/ConTexto/artic...,ConTexto


In [15]:
df.to_csv('../data/CONTEXTO.csv', index=False)

In [16]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2016-03-18,ANÁLISE DE SUSTENTABILIDADE AMBIENTAL: ESTUDO ...,As organizações conquistam desenvolvimento sus...,Indústria de eletrodomésticos. Sustentabilidad...,4,Daniela Di Domenico,Sady Mazzioni,Maicon Andre Provin,Luciano Toniello,,,Universidade Comunitária da Região de Chapecó ...,Universidade Comunitária da Região de Chapecó ...,Universidade Comunitária da Região de Chapecó ...,Universidade Comunitária da Região de Chapecó ...,,,https://seer.ufrgs.br/index.php/ConTexto/artic...,ConTexto
1,2019-06-27,MAPEAMENTO DA PESQUISA SOBRE DISCLOSURE AMBIEN...,O objetivo desse trabalho é apresentar uma rev...,Disclosure ambiental. Revisão da literatura. R...,2,Márcio Abadio Ramos,Rodrigo Fernandes Malaquias,,,,,Universidade Federal de Uberlândia.,Universidade Federal de Uberlândia,,,,,https://seer.ufrgs.br/index.php/ConTexto/artic...,ConTexto
2,2019-11-25,ANÁLISE DAS DEMONSTRAÇÕES CONTÁBEIS: UMA PROPO...,Este trabalho apresenta os intervalos dos indi...,Demonstrações Contábeis. Análise. Índices de L...,3,Paula Hortência da Silva Soares,Juliano Almeida de Faria,José Jackson de Oliveira,,,,Universidade Estadual de Feira de Santana Aven...,Universidade Estadual de Feira de Santana Aven...,Universidade Estadual de Feira de Santana Univ...,,,,https://seer.ufrgs.br/index.php/ConTexto/artic...,ConTexto
3,2020-05-22,NÍVEL DE EVIDENCIAÇÃO DAS EMPRESAS DO SETOR DE...,O objetivo do estudo é identificar o nível de ...,NBC T 15. Disclosure. contabilidade ambiental.,3,Nathasha Gonçalves Goulart de Oliveira,Fabíola Graciele Besen,Valdir Serafim Junior,,,,Aluna do Curso de Ciências Contábeis da Unioes...,Unioeste - Universidade Estadual do Oeste do P...,Unioeste - Universidade Estadual do Oeste do P...,,,,https://seer.ufrgs.br/index.php/ConTexto/artic...,ConTexto
4,2014-08-30,EVIDENCIAÇÃO CONTÁBIL EM ENTIDADES BRASILEIRAS...,Este estudo objetiva identificar a adequação d...,Evidenciação. Prestação de contas. Terceiro se...,3,Vanderlei Gollo,Sheila Jeane Schulz,Fabricia Silva Da Rosa,,,,PPGCC/FURB - Programa de Pós-Graduação da Univ...,PPGCC/FURB - Programa de Pós-Graduação da Univ...,PPGCC/FURB - Programa de Pós-Graduação da Univ...,,,,https://seer.ufrgs.br/index.php/ConTexto/artic...,ConTexto
5,2014-04-29,UMA DISCUSSÃO CONCEITUAL SOBRE O TRATAMENTO CO...,Heritage assets (HAs) são ativos físicos prese...,Ativos hereditários. Bens de uso comum. Teoria...,4,Orleans Silva Martins,Antônio Maria Henri Beyle de Araújo,Diana Vaz de Lima,Jorge Katsumi Niyama,,,Universidade Federal da Paraíba,Banco Central do Brasil,Universidade de Brasília,Universidade de Brasília,,,https://seer.ufrgs.br/index.php/ConTexto/artic...,ConTexto
6,2014-08-30,EVIDENCIAÇÃO AMBIENTAL: UM ESTUDO A PARTIR DE ...,"Sintonizados com os acontecimentos mundiais, p...",Evidenciação. Divulgação ambiental. Demonstraç...,5,Fábia Jaiany Viana de Souza,Célio da Costa Barros,Renata Paes de Barros Camara,Fábio Resende de Araújo,Mauricio Correa da Silva,,Programa Multi-institucional e Inter-Regional ...,Programa Multi-institucional e Inter-Regional ...,Universidade de São Paulo (USP).,Universidade Federal do Rio Grande do Norte (U...,Universidade Federal do Rio Grande do Norte,,https://seer.ufrgs.br/index.php/ConTexto/artic...,ConTexto
7,2014-04-29,CONTABILIZAÇÃO DE RESÍDUOS GERADOS NA ATIVIDAD...,Sabendo-se que os profissionais ainda são relu...,Gestão contábil ambiental. Resíduos. Contabili...,2,Ana Carolina Vasconcelos Colares,Márcia Athayde Matias,,,,,Pontifícia Universidade Católica de Minas Gerais,UFMG,,,,,https://seer.ufrgs.br/index.php/ConTexto/artic...,ConTexto
8,2021-05-13,IMPACTO DA GOVERNANÇA CORPORATIVA NO DISCLOSUR...,O estudo teve como objetivo principal verifica...,Passivos Contingentes,2,Alex Ferreira Lopes,Thamirys de Sousa Correia,,,,,Universidade Federal da Paraíba,Universida